In [ ]:
#1. read this file as a csv "uniprot_sequences_with_deletions.csv"
#2. crop the UniProt_sequence based on start and end indices
#3. for the sequences Generate the features using iFeatureOmega
#4. save features - seperated- inside "features" folder

In [ ]:
# Step 1: Install all required packages
!pip install biopython networkx iFeatureOmegaCLI


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 71.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 24.0 MB/s eta 0:00:00
  Created wheel for matplotlib: filename=matplotlib-3.4.3-cp311-cp311-linux_x86_64.whl size=10427109 sha256=0643f9cb7c50e266e37ea31b79089e438fd6011297c19f5f0738edca7116adcc
  Stored in directory: /root/.cache/pip/wheels/5a/99/fa/d6b0fc4a7c0f444b5f0ee8e7513f1d3095338bc1b458033e5d
Successfully built matplotlib
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires matplotlib>=3.7.1, but you have matplo

In [ ]:
!pip show iFeatureOmegaCLI

Name: iFeatureOmegaCLI
Version: 1.0.2
Summary: An integrative platform for the prediction/feature engineering, visualization and analysis of features from molecular sequence, structural and ligand data sets
Home-page: https://github.com/Superzchen/iFeatureOmega-CLI
Author: SuperZhen
Author-email: chenzhen-win2009@163.com
License: MIT License
Location: /usr/local/lib/python3.11/dist-packages
Requires: matplotlib, networkx, numpy, pandas, scikit-learn, scipy
Required-by: 


In [ ]:
!pip install numpy==1.24.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 138.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.5.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
bigframes 2.8.0 requires matplotlib>=3.7.1, but you have matplotlib 3.4.3 which is incompatible.
arviz 0.21.0 requires matplotlib>=3.5, but you have matplotlib 3.4.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2;

In [ ]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 89.6 MB/s eta 0:00:00


In [ ]:
# Step 2: Import necessary libraries
import pandas as pd
import numpy as np
np.float = float  # Patch for compatibility

import scipy

# Patch: assign numpy.triu to scipy.triu
scipy.triu = np.triu
scipy.argwhere = np.argwhere


import os
from tqdm import tqdm
import iFeatureOmegaCLI
from rdkit import Chem

In [ ]:
# Step 3: Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 4: Define paths and parameters
input_csv_path = "/content/drive/MyDrive/Kd_Meshari/data/kiba/proteins.txt"
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"
combined_sequences_file = os.path.join(output_features_dir, "all_sequences1.fasta")

# Create output directory if it doesn't exist
os.makedirs(output_features_dir, exist_ok=True)

import json

# Load JSON file
with open(input_csv_path, "r") as f:
    df = json.load(f)

# Write to FASTA format
with open(combined_sequences_file, "w") as fasta_file:
    for uniprot_id, sequence in df.items():
        fasta_file.write(f">{uniprot_id}\n")
        # Wrap the sequence at 60 characters per line (FASTA convention)
        for i in range(0, len(sequence), 60):
            fasta_file.write(sequence[i:i+60] + "\n")

In [ ]:
# Step 4: Define paths and parameters
input_csv_path = "/content/drive/MyDrive/Kd_Meshari/proccessed_kiba/combined_data.csv"
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"
combined_sequences_file = os.path.join(output_features_dir, "all_sequences.fasta")

# Create output directory if it doesn't exist
os.makedirs(output_features_dir, exist_ok=True)

# Step 5: Read the CSV file and create combined FASTA file
df = pd.read_csv(input_csv_path)

print("Creating combined sequence file...")
with open(combined_sequences_file, 'w') as fasta_file:
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Get the sequence and crop it
        full_sequence = row['UniProt_sequence']
        start = int(row['start']) - 1  # convert to 0-based index
        end = int(row['end'])
        #end = start + 20
        cropped_sequence = full_sequence[start:end]

        # Create sequence ID (gene + uniprot + indices)
        seq_id = f">{row['gene']}_{row['uniprot_accession']}_{row['start']}_{end}"

        # Write to FASTA file
        fasta_file.write(f"{seq_id}\n{cropped_sequence}\n")

print(f"Combined sequences saved to: {combined_sequences_file}")

Creating combined sequence file...


100%|██████████| 229/229 [00:00<00:00, 14704.01it/s]

Combined sequences saved to: /content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq/all_sequences.fasta


In [ ]:

# Step 6: Define the descriptors we want to use

descriptors = [
    "AAC", "EAAC", "CKSAAP type 1", "CKSAAP type 2", "DPC type 1", "DPC type 2",
     "CTDC", "CTDT", "CTDD", "CTriad", "KSCTriad", "TPC type 1", "TPC type 2",
    "ASDC", "DistancePair", "GAAC", "EGAAC", "CKSAAGP type 1", "CKSAAGP type 2",
    "GDPC type 1", "GDPC type 2", "GTPC type 1", "GTPC type 2", "Moran", "Geary",
    "NMBroto", "AC", "CC", "ACC", "SOCNumber", "QSOrder", "PAAC", "APAAC",
    "PseKRAAC type 1", "PseKRAAC type 2", "PseKRAAC type 3A", "PseKRAAC type 3B",
    "PseKRAAC type 4", "PseKRAAC type 5", "PseKRAAC type 6A", "PseKRAAC type 6B",
    "PseKRAAC type 6C", "PseKRAAC type 7", "PseKRAAC type 8", "PseKRAAC type 9",
    "PseKRAAC type 10", "PseKRAAC type 11", "PseKRAAC type 12", "PseKRAAC type 13",
    "PseKRAAC type 14", "PseKRAAC type 15", "PseKRAAC type 16", "binary",
    "binary_6bit", "binary_5bit type 1", "binary_5bit type 2", "binary_3bit type 1",
    "binary_3bit type 2", "binary_3bit type 3", "binary_3bit type 4",
    "binary_3bit type 5", "binary_3bit type 6", "binary_3bit type 7",
    "AESNN3", "OPF_10bit", "OPF_7bit type 1", "OPF_7bit type 2", "OPF_7bit type 3",
    "AAIndex", "BLOSUM62", "ZScale", "KNN"
]


In [ ]:
# Step 7: Initialize iProtein with the combined sequence file
print("\nInitializing feature calculation...")
protein = iFeatureOmegaCLI.iProtein(combined_sequences_file)
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"
# Step 8: Calculate and save features for each descriptor
successful_descriptors = 0
total_feature_count = 0
descriptor_feature_map = {}

for desc in tqdm(descriptors):
    try:
        print(f"\nCalculating: {desc}")
        protein.get_descriptor(desc)

        if protein.encodings is None or protein.encodings.empty:
            raise ValueError("Encoding is None or empty.")

        num_features = protein.encodings.shape[1]
        print(f"Feature count: {num_features}")

        total_feature_count += num_features
        successful_descriptors += 1
        descriptor_feature_map[desc] = num_features

        # Save features to CSV
        filename = desc.replace(" ", "_").replace("/", "_") + ".csv"
        output_path = os.path.join(output_features_dir, filename)
        protein.to_csv(output_path, index=False, header=True)
        print(f"Saved: {filename}")

    except Exception as e:
        print(f"Skipped {desc}: {str(e)}")
        continue

# Step 9: Save summary file
summary_path = os.path.join(output_features_dir, "feature_generation_summary.txt")
with open(summary_path, 'w') as f:
    f.write("Feature Generation Summary\n")
    f.write("=========================\n\n")
    f.write(f"Total sequences processed: {len(df)}\n")
    f.write(f"Total descriptors attempted: {len(descriptors)}\n")
    f.write(f"Total descriptors succeeded: {successful_descriptors}\n")
    f.write(f"Total features generated: {total_feature_count}\n\n")
    f.write("Successful descriptors with feature counts:\n")
    for desc, count in descriptor_feature_map.items():
        f.write(f"{desc}: {count} features\n")

print("\nProcess completed successfully!")
print(f"All files saved to: {output_features_dir}")
print(f"Combined sequences: {combined_sequences_file}")
print(f"Summary file: {summary_path}")


Initializing feature calculation...


  0%|          | 0/72 [00:00<?, ?it/s]


Calculating: AAC
Feature count: 20


  1%|▏         | 1/72 [00:00<00:33,  2.12it/s]

Saved: AAC.csv

Calculating: EAAC
Skipped EAAC: Encoding is None or empty.

Calculating: CKSAAP type 1
Feature count: 1600


...

 99%|█████████▊| 71/72 [02:00<00:00,  3.05it/s]

Saved: ZScale.csv

Calculating: KNN
Feature count: 4


100%|██████████| 72/72 [02:01<00:00,  1.68s/it]

Saved: KNN.csv



Process completed successfully!
All files saved to: /content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq
Combined sequences: /content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq/all_sequences.fasta
Summary file: /content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq/feature_generation_summary.txt


In [ ]:
!pip install numpy==1.24.4 torch transformers biopython --upgrade --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 130.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 261.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 325.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 258.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 285.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 205.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 383.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 270.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 299.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
print("✅ NumPy and transformers are working:", np.__version__)

✅ NumPy and transformers are working: 1.24.4


In [ ]:
!pip install biopython

In [ ]:
from transformers import AutoTokenizer, AutoModel
from Bio import SeqIO
import torch
import pandas as pd
import os
from tqdm import tqdm

# Paths
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"

fasta_file = os.path.join(output_features_dir, "all_sequences.fasta")
output_csv = "/content/drive/MyDrive/Kd_Meshari/features/ProtBERT_embeddings.csv"

# Load ProtBERT
model_name = "Rostlab/prot_bert_bfd"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
model = AutoModel.from_pretrained(model_name)
model = model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# Embed a single sequence
def embed_sequence(seq):
    seq = ' '.join(list(seq))  # Add spaces between amino acids
    inputs = tokenizer(seq, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        output = model(**inputs)
    embeddings = output.last_hidden_state.squeeze(0)
    return embeddings.mean(dim=0).cpu().numpy()  # Mean-pooled [1024-d]

# Extract from FASTA
embeddings = []
for record in tqdm(SeqIO.parse(fasta_file, "fasta")):
    try:
        emb = embed_sequence(str(record.seq))
        embeddings.append([record.id] + emb.tolist())
    except Exception as e:
        print(f"Failed for {record.id}: {e}")

# Save to CSV
df = pd.DataFrame(embeddings)
df.columns = ["ProteinID"] + [f"ProtBERT_{i}" for i in range(1024)]
df.to_csv(output_csv, index=False)
print(f"\n✅ Saved ProtBERT embeddings to:\n{output_csv}")

0it [00:00, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
229it [04:23,  1.15s/it]



✅ Saved ProtBERT embeddings to:
/content/drive/MyDrive/Kd_Meshari/features/ProtBERT_embeddings.csv


In [ ]:
!pip install jax-unirep jax jaxlib biopython --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 15.3 MB/s eta 0:00:00


In [ ]:
from jax_unirep import get_reps
from Bio import SeqIO
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# Paths
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"

fasta_file = os.path.join(output_features_dir, "all_sequences.fasta")
output_csv = "/content/drive/MyDrive/Kd_Meshari/features/UniRep_jax_embeddings.csv"
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

# Read sequences
records = list(SeqIO.parse(fasta_file, "fasta"))

# Embed all at once (batched)
sequences = [str(rec.seq) for rec in records]
h_avg, h_final, c_final = get_reps(sequences)  # h_avg shape = (N, 1900)

# Build DataFrame
df = pd.DataFrame(h_avg, columns=[f"UniRep_{i}" for i in range(h_avg.shape[1])])
df.insert(0, "ProteinID", [rec.id for rec in records])
df.to_csv(output_csv, index=False)

print(f"✅ Saved UniRep embeddings (batched) to {output_csv}")

✅ Saved UniRep embeddings (batched) to /content/drive/MyDrive/Kd_Meshari/features/UniRep_jax_embeddings.csv


In [ ]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 745.9 kB/s eta 0:00:00


In [ ]:
# Step 4: Define paths and parameters
input_csv_path = "/content/drive/MyDrive/Kd_Meshari/proccessed_kiba/domain_binding_site_summary_updated_manually.csv"
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"
combined_sequences_file = os.path.join(output_features_dir, "all_sequences_binding_sites.fasta")

# Create output directory if it doesn't exist
os.makedirs(output_features_dir, exist_ok=True)

# Step 5: Read the CSV file and create combined FASTA file
df = pd.read_csv(input_csv_path)

# Step 2: Split the 'binding_site_span' column into start and end
df[['span_start', 'span_end']] = df['binding_site_span'].astype(str).str.split('--', expand=True)
df['span_start'] = pd.to_numeric(df['span_start'], errors='coerce')
df['span_end'] = pd.to_numeric(df['span_end'], errors='coerce')


# Step 5: Write cropped sequences to FASTA
with open(combined_sequences_file, 'w') as fasta_file:
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        full_sequence = str(row['UniProt_sequence'])

        # Validate start/end positions
        if pd.notna(row['span_start']) and pd.notna(row['span_end']):
            start = int(row['span_start']) - 1  # 0-based index
            end = int(row['span_end'])

            if 0 <= start < end <= len(full_sequence):
                cropped_sequence = full_sequence[start:end]
                seq_id = f">{row['gene']}_{row['uniprot_accession']}_{row['span_start']}_{row['span_end']}"
                fasta_file.write(f"{seq_id}\n{cropped_sequence}\n")

print(f"Combined FASTA written to: {combined_sequences_file}")

100%|██████████| 229/229 [00:00<00:00, 17315.27it/s]

Combined FASTA written to: /content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq/all_sequences_binding_sites.fasta


In [ ]:
import esm
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


In [ ]:
import torch
from transformers import T5Tokenizer, T5EncoderModel
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO

def extract_prott5_embeddings_to_csv(fasta_file, output_csv="protT5_embeddings.csv", device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Load ProtT5 model and tokenizer
    tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50").to(device)
    model.eval()

    # Read FASTA file
    records = list(SeqIO.parse(fasta_file, "fasta"))

    # Prepare storage
    all_embeddings = []
    sequence_ids = []

    for record in tqdm(records, desc="Extracting ProtT5 embeddings"):
        try:
            seq_id = record.id
            seq = str(record.seq).replace(" ", "").upper()
            seq = ' '.join(list(seq))  # spacing for ProtT5 tokenizer

            # Tokenize and get embeddings
            inputs = tokenizer(seq, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                embedding = outputs.last_hidden_state[0].mean(dim=0).cpu().numpy()

            # Store
            sequence_ids.append(seq_id)
            all_embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing {record.id}: {e}")
            continue

    # Save to CSV
    df = pd.DataFrame(all_embeddings)
    df.insert(0, "sequence_id", sequence_ids)
    df.to_csv(output_csv, index=False)
    return output_csv



# Paths
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"

fasta_file = os.path.join(output_features_dir, "all_sequences.fasta")
output_csv = "/content/drive/MyDrive/Kd_Meshari/features/protein_embeddings.csv"
extract_prott5_embeddings_to_csv(fasta_file, output_csv)


Extracting ProtT5 embeddings: 100%|██████████| 229/229 [14:34<00:00,  3.82s/it]


'/content/drive/MyDrive/Kd_Meshari/features/protein_embeddings.csv'

In [ ]:
import torch
import esm
from Bio import SeqIO
import pandas as pd
from tqdm import tqdm
import os

output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq"

device = "cuda" if torch.cuda.is_available() else "cpu"

input_fasta = os.path.join(output_features_dir, "all_sequences.fasta")
output_dir = output_features_dir
os.makedirs(output_dir, exist_ok=True)

# Load models
esm2_model, esm2_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
esm1b_model, esm1b_alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
esm2_batch_converter = esm2_alphabet.get_batch_converter()
esm1b_batch_converter = esm1b_alphabet.get_batch_converter()

esm2_model = esm2_model.to(device).eval()
esm1b_model = esm1b_model.to(device).eval()

# Read sequences
records = list(SeqIO.parse(input_fasta, "fasta"))

def extract_esm_embeddings(model, converter, records, model_name):
    embeddings = []
    for record in tqdm(records, desc=f"Processing {model_name}"):
        try:
            data = [(record.id, str(record.seq))]
            batch_labels, batch_strs, batch_tokens = converter(data)
            batch_tokens = batch_tokens.to(device)

            with torch.no_grad():
                results = model(batch_tokens, repr_layers=[33], return_contacts=False)
            token_representations = results["representations"][33]

            seq_len = len(record.seq)
            emb = token_representations[0, 1:seq_len+1].mean(0).cpu().numpy()
            emb_dict = {f"{model_name}_feat_{i}": val for i, val in enumerate(emb)}
            emb_dict["sequence_id"] = record.id
            embeddings.append(emb_dict)
        except Exception as e:
            print(f"Error processing {record.id}: {e}")
            continue

    df = pd.DataFrame(embeddings)
    df.to_csv(os.path.join(output_dir, f"{model_name}_embeddings.csv"), index=False)

# Run both models
extract_esm_embeddings(esm2_model, esm2_batch_converter, records, "esm2")
extract_esm_embeddings(esm1b_model, esm1b_batch_converter, records, "esm1b")

print("✅ Embeddings saved to:", output_dir)


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt
Processing esm1b: 100%|██████████| 229/229 [07:43<00:00,  2.03s/it]


✅ Embeddings saved to: /content/drive/MyDrive/Kd_Meshari/features/Kiba_features_from_domain_seq


In [ ]:
!grep -R "GetNumAtoms(" /usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/

/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/AtomTypes.py:    nAtoms = mol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/property.py:    nheavy = mol.GetNumAtoms(onlyHeavy=1)
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/burden.py:    Natom = mol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/basak.py:    nAtoms = Hmol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/basak.py:    nAtoms = Hmol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/basak.py:    nAtoms = Hmol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/basak.py:        nAtoms = Hmol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/basak.py:        nAtoms = Hmol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/chem/basak.py:        nAtoms = Hmol.GetNumAtoms()
/usr/local/lib/python3.11/dist-packages/iFeatu

In [ ]:
!find /usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/ -name "*.py" -exec sed -i 's/\([A-Za-z0-9_]*\)\.GetNumAtoms(onlyHeavy=1)/\1.GetNumHeavyAtoms()/g' {} +

In [ ]:
!grep -R "GetNumAtoms(onlyHeavy=" /usr/local/lib/python3.11/dist-packages/iFeatureOmegaCLI/




---



# Ligands features

In [ ]:
import json

original_ligands_path = "/content/drive/MyDrive/Kd_Meshari/data/kiba/ligands_can.txt"
input_smiles_path = "/content/drive/MyDrive/Kd_Meshari/features/kiba_ligands.txt"

# Load the JSON
with open(original_ligands_path, 'r') as f:
    ligands_dict = json.load(f)

# Write only SMILES strings to new file
with open(input_smiles_path, 'w') as f:
    for smiles in ligands_dict.values():
        f.write(smiles + '\n')

print("✅ SMILES-only file written to:", input_smiles_path)


✅ SMILES-only file written to: /content/drive/MyDrive/Kd_Meshari/features/kiba_ligands.txt


In [ ]:
output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_ligand_features"
largest_fragments = "/content/drive/MyDrive/Kd_Meshari/features/kiba_ligands_processed.txt"
# Create output directory
os.makedirs(output_features_dir, exist_ok=True)

# Step 5: Define feature types
feature_types = [
    "Basak",
    "Burden",
    "Pharmacophore",
    "Constitution",
    "Topology",
    "Connectivity",
    "Kappa",
    "EState",
    "Autocorrelation-moran",
    "Autocorrelation-geary",
    "Autocorrelation-broto",
    "Molecular properties",
    "Charge",
    "Moe-Type descriptors",
    "MACCS fingerprints",
    "Morgan-ECFP4 fingerprint",
    "Morgan-ECFP6 fingerprint",
    "Morgan-FCFP4 fingerprint",
    "Morgan-FCFP6 fingerprint",
    "E-state fingerprints"
]


def get_largest_fragment_smiles(smiles: str) -> str:
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fragments = Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=True)
    largest = max(fragments, key=lambda m: m.GetNumAtoms())
    return Chem.MolToSmiles(largest)

def process_smiles_file(input_path: str, output_path: str):
    with open(input_path, 'r') as infile:
        smiles_list = infile.read().splitlines()

    cleaned_smiles = []
    for smi in smiles_list:
        largest = get_largest_fragment_smiles(smi)
        if largest:
            cleaned_smiles.append(largest)

    with open(output_path, 'w') as outfile:
        outfile.write("\n".join(cleaned_smiles))

# Example usage:
process_smiles_file(input_smiles_path, largest_fragments)


# Step 6: Initialize ligand feature calculator
print("Loading SMILES file...")
ligand = iFeatureOmegaCLI.iLigand(largest_fragments)

# Step 7: Process each descriptor
total_features = 0
processed_descriptors = 0
successful_descriptors = {}

for desc in tqdm(feature_types):
    try:
        print(f"\nCalculating: {desc}")
        ligand.get_descriptor(desc)

        if ligand.encodings is None or ligand.encodings.empty:
            raise ValueError("No features returned")

        num_features = ligand.encodings.shape[1]
        print(f"Feature count: {num_features}")

        # Save features to CSV
        filename = desc.replace(" ", "_").replace("-", "_").lower() + ".csv"
        output_path = os.path.join(output_features_dir, filename)
        ligand.to_csv(output_path, index=False, header=True)
        print(f"Saved: {filename}")

        total_features += num_features
        processed_descriptors += 1
        successful_descriptors[desc] = num_features

    except Exception as e:
        print(f"Skipped {desc}: {str(e)}")
        continue

# Step 8: Save summary file
summary_path = os.path.join(output_features_dir, "ligand_feature_summary.txt")
with open(summary_path, 'w') as f:
    f.write("Ligand Feature Generation Summary\n")
    f.write("=================================\n\n")
    f.write(f"Input SMILES file: {input_smiles_path}\n")
    f.write(f"Total descriptors attempted: {len(feature_types)}\n")
    f.write(f"Total descriptors succeeded: {processed_descriptors}\n")
    f.write(f"Total features generated: {total_features}\n\n")
    f.write("Successful descriptors with feature counts:\n")
    for desc, count in successful_descriptors.items():
        f.write(f"{desc}: {count} features\n")

print("\nProcessing complete!")
print(f"All feature files saved to: {output_features_dir}")
print(f"Summary file: {summary_path}")

Loading SMILES file...


  0%|          | 0/20 [00:00<?, ?it/s]


Calculating: Basak
Feature count: 21


  5%|▌         | 1/20 [10:40<3:22:42, 640.11s/it]

Saved: basak.csv

Calculating: Burden
Feature count: 64


...

 95%|█████████▌| 19/20 [26:38<00:15, 15.71s/it]

Saved: morgan_fcfp6_fingerprint.csv

Calculating: E-state fingerprints
Feature count: 79


100%|██████████| 20/20 [27:15<00:00, 81.78s/it]

Saved: e_state_fingerprints.csv



Processing complete!
All feature files saved to: /content/drive/MyDrive/Kd_Meshari/features/Kiba_ligand_features
Summary file: /content/drive/MyDrive/Kd_Meshari/features/Kiba_ligand_features/ligand_feature_summary.txt


In [ ]:
!pip install rdkit-pypi
!apt-get install -y openbabel
!pip install openbabel-wheel

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7
The following NEW packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7 openbabel
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 3,903 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinchi1 amd64 1.03+dfsg-4 [455 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmaeparser1 amd64 1.2.4-1build1 [88.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenbabel7 amd64 3.1.1+dfsg-6ubuntu5 [3,231 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openbabel amd64 3.1.1+dfsg-6ubuntu5 [128 kB]
Fetched 3,903 kB in 3s (1,494 kB/s)
Selecting previously unselected package libinchi1.
(Reading database ... 126281 files and dire

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from openbabel import pybel
# import numpy as np
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import rdmolops

def get_pubchem_like_fingerprint(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")

        # Select largest fragment
        frags = Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=True)
        mol = max(frags, key=lambda m1: m1.GetNumAtoms())

        # Get RDKFingerprint (defaults to 2048 bits)
        fp = rdmolops.RDKFingerprint(mol)

        # Convert to dictionary
        return {f'pubchem_like_fp_{i}': int(fp.GetBit(i)) for i in range(fp.GetNumBits())}

    except Exception as e:
        print(f"Fingerprint error for {smiles}: {e}")
        return {}

def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")

        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
        return {f'morgan_fp_{i}': int(fp.GetBit(i)) for i in range(nBits)}

    except Exception as e:
        print(f"Morgan FP error for {smiles}: {e}")
        return {}

def get_bitstring_descriptors(smiles):
    if isinstance(smiles, str) and smiles.strip():
        mol = pybel.readstring('smi', smiles.strip())
    else:
        raise ValueError("Invalid SMILES input")
    mol.OBMol.AddHydrogens()
    fps = mol.calcfp("FP4").bits
    bitstring = list('0' * 307)
    for item in fps:
        bitstring[item - 1] = '1'
    return {f'bitstring{i}': int(bitstring[i]) for i in range(len(bitstring))}


def get_maccs_keys(smiles):
    mol = Chem.MolFromSmiles(smiles)
    frags = Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=True)
    mol = max(frags, key=lambda m1: m1.GetNumAtoms())
    try:
        fps_macs1 = MACCSkeys.GenMACCSKeys(mol).ToBitString()
        return {f'macs_key{i}': fps_macs1[i] for i in range(len(fps_macs1))}
    except Exception as e:
        print(f"MACCS error for: {smiles}\n{e}")
        return {}


def get_chemical_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    frags = Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=True)
    mol = max(frags, key=lambda m1: m1.GetNumAtoms())
    chem_desc = np.zeros(190)
    try:
        D = Descriptors
        descriptor_functions = [
            D.MaxEStateIndex, D.MinEStateIndex, D.MaxAbsEStateIndex, D.MinAbsEStateIndex,
            D.BalabanJ, D.BertzCT, D.Chi0, D.Chi0n, D.Chi0v, D.Chi1, D.Chi1n, D.Chi1v,
            D.Chi2n, D.Chi2v, D.Chi3n, D.Chi3v, D.Chi4n, D.Chi4v,
            D.EState_VSA1, D.EState_VSA2, D.EState_VSA3, D.EState_VSA4, D.EState_VSA5,
            D.EState_VSA6, D.EState_VSA7, D.EState_VSA8, D.EState_VSA9, D.EState_VSA10,
            D.EState_VSA11, D.FractionCSP3, D.HallKierAlpha, D.HeavyAtomCount, D.Ipc,
            D.Kappa1, D.Kappa2, D.Kappa3, D.LabuteASA, D.MolLogP, D.MolMR, D.NHOHCount,
            D.NOCount, D.NumAliphaticCarbocycles, D.NumAliphaticHeterocycles,
            D.NumAliphaticRings, D.NumAromaticCarbocycles, D.NumAromaticHeterocycles,
            D.NumAromaticRings, D.NumHAcceptors, D.NumHDonors, D.NumHeteroatoms,
            D.NumRotatableBonds, D.NumSaturatedCarbocycles, D.NumSaturatedHeterocycles,
            D.PEOE_VSA1, D.PEOE_VSA10, D.PEOE_VSA11, D.PEOE_VSA12, D.PEOE_VSA13,
            D.PEOE_VSA14, D.PEOE_VSA2, D.PEOE_VSA3, D.PEOE_VSA4, D.PEOE_VSA5,
            D.PEOE_VSA6, D.PEOE_VSA7, D.PEOE_VSA8, D.PEOE_VSA9, D.RingCount,
            D.SMR_VSA1, D.SMR_VSA10, D.SMR_VSA2, D.SMR_VSA3, D.SMR_VSA4, D.SMR_VSA5,
            D.SMR_VSA6, D.SMR_VSA7, D.SMR_VSA8, D.SMR_VSA9,
            D.SlogP_VSA1, D.SlogP_VSA10, D.SlogP_VSA11, D.SlogP_VSA12, D.SlogP_VSA2,
            D.SlogP_VSA3, D.SlogP_VSA4, D.SlogP_VSA5, D.SlogP_VSA6, D.SlogP_VSA7,
            D.SlogP_VSA8, D.SlogP_VSA9,
            D.TPSA,
            D.VSA_EState1, D.VSA_EState10, D.VSA_EState2, D.VSA_EState3, D.VSA_EState4,
            D.VSA_EState5, D.VSA_EState6, D.VSA_EState7, D.VSA_EState8, D.VSA_EState9,
            D.fr_Al_COO, D.fr_Al_OH, D.fr_Al_OH_noTert, D.fr_ArN, D.fr_Ar_COO,
            D.fr_Ar_N, D.fr_Ar_NH, D.fr_Ar_OH, D.fr_COO, D.fr_COO2, D.fr_C_O,
            D.fr_C_O_noCOO, D.fr_C_S, D.fr_HOCCN, D.fr_Imine, D.fr_NH0, D.fr_NH1,
            D.fr_NH2, D.fr_N_O, D.fr_Ndealkylation1, D.fr_Ndealkylation2,
            D.fr_Nhpyrrole, D.fr_SH, D.fr_aldehyde, D.fr_alkyl_carbamate,
            D.fr_alkyl_halide, D.fr_allylic_oxid, D.fr_amide, D.fr_amidine,
            D.fr_aniline, D.fr_aryl_methyl, D.fr_azide, D.fr_azo, D.fr_barbitur,
            D.fr_benzene, D.fr_benzodiazepine, D.fr_bicyclic, D.fr_diazo,
            D.fr_dihydropyridine, D.fr_epoxide, D.fr_ester, D.fr_ether, D.fr_furan,
            D.fr_guanido, D.fr_halogen, D.fr_hdrzine, D.fr_hdrzone, D.fr_imidazole,
            D.fr_imide, D.fr_isocyan, D.fr_isothiocyan, D.fr_ketone,
            D.fr_ketone_Topliss, D.fr_lactam, D.fr_lactone, D.fr_methoxy,
            D.fr_morpholine, D.fr_nitrile, D.fr_nitro, D.fr_nitro_arom,
            D.fr_nitro_arom_nonortho, D.fr_nitroso, D.fr_oxazole, D.fr_oxime,
            D.fr_para_hydroxylation, D.fr_phenol, D.fr_phenol_noOrthoHbond,
            D.fr_phos_acid, D.fr_phos_ester, D.fr_piperdine, D.fr_piperzine,
            D.fr_priamide, D.fr_prisulfonamd, D.fr_pyridine, D.fr_quatN,
            D.fr_sulfide, D.fr_sulfonamd, D.fr_sulfone, D.fr_term_acetylene,
            D.fr_tetrazole, D.fr_thiazole, D.fr_thiocyan, D.fr_thiophene,
            D.fr_unbrch_alkane, D.fr_urea,
            D.MolWt, D.HeavyAtomMolWt, D.NumValenceElectrons, D.NumSaturatedRings
        ]
        for i, func in enumerate(descriptor_functions):
            chem_desc[i] = func(mol)
        chem_desc = np.round(chem_desc, 4).tolist()
        return {f'chem_des{i}': chem_desc[i] for i in range(len(chem_desc))}
    except Exception as e:
        print(f"Chemical descriptor error for: {smiles}\n{e}")
        return {}

In [ ]:
# burden.csv and constitution.csv are missing

In [ ]:
smiles = "CC1CCN(CC1N(C)C2=NC=NC3=C2C=CN3)C(=O)CC#N"

bitstring_desc = get_bitstring_descriptors(smiles)
maccs_keys = get_maccs_keys(smiles)
chemical_desc = get_chemical_descriptors(smiles)
fp_dict = get_pubchem_like_fingerprint(smiles)
print(list(fp_dict.items())[:10])  # First 10 bits

print(type(bitstring_desc))
print(type(maccs_keys))
print(type(chemical_desc))
print(len(fp_dict))
print(len(bitstring_desc))
print(len(maccs_keys))
print(len(chemical_desc))

[('pubchem_like_fp_0', 1), ('pubchem_like_fp_1', 1), ('pubchem_like_fp_2', 1), ('pubchem_like_fp_3', 0), ('pubchem_like_fp_4', 0), ('pubchem_like_fp_5', 1), ('pubchem_like_fp_6', 1), ('pubchem_like_fp_7', 0), ('pubchem_like_fp_8', 0), ('pubchem_like_fp_9', 1)]
<class 'dict'>
<class 'dict'>
<class 'dict'>
2048
307
167
190


In [ ]:
def process_ligands(LIGAND_TXT_PATH):
    print("Processing ligands...")

    # Read SMILES
    with open(LIGAND_TXT_PATH) as f:
        smiles_list = [line.strip() for line in f if line.strip()]

    all_descriptors = []
    failed_molecules = []

    for smi in tqdm(smiles_list):
        try:
            mol_rdkit = Chem.MolFromSmiles(smi)
            print(smi)
            # Remove disconnected fragments
            frags = Chem.GetMolFrags(mol_rdkit, asMols=True, sanitizeFrags=True)
            mol_rdkit = max(frags, key=lambda m: m.GetNumAtoms())
            if not mol_rdkit:
                raise ValueError("Invalid SMILES")

            mol_descriptors = {'smiles': smi}

            # Custom descriptor functions
            bitstring_desc = get_bitstring_descriptors(smi)
            macs_desc = get_maccs_keys(smi)
            chemical_desc = get_chemical_descriptors(smi)
            pubchem_fp = get_pubchem_like_fingerprint(smi)

            mol_descriptors.update(bitstring_desc)
            mol_descriptors.update(pubchem_fp)
            #mol_descriptors.update(macs_desc)
            mol_descriptors.update(chemical_desc)

            all_descriptors.append(mol_descriptors)
        except Exception as e:
            failed_molecules.append((smi, str(e)))
            continue
    print("Here: ", len(all_descriptors))
    # Save results
    if all_descriptors:
        ligand_df = pd.DataFrame(all_descriptors)
        output_path = os.path.join(output_features_dir, "Babel_Chemicals/ligand_descriptors.csv")
        ligand_df.to_csv(output_path, index=False)
        print(f"Saved ligand features to {output_path}")
        print(ligand_df.shape)

    # Save failure log
    if failed_molecules:
        with open(os.path.join(output_features_dir, "Babel_Chemicals/failed_molecules.log"), 'w') as f:
            for smi, error in failed_molecules:
                f.write(f"{smi}: {error}\n")

In [ ]:
# ==============================================
# Main Execution
# ==============================================
if __name__ == "__main__":
    process_ligands(input_smiles_path)
    print("Feature extraction complete!")

Processing ligands...


  0%|          | 6/2111 [00:00<00:39, 53.66it/s]

COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl
COC1=C(C=C2C(=C1)CCN=C2C3=CC(=CC=C3)Cl)Cl
C1COCCN1C2=CC(=CC=C2)NC3=NC=CC(=N3)C4=C(N=C5N4C=CS5)C6=CC(=CC=C6)NC(=O)CC7=CC=CC=C7
C1=CC2=C(C=C1C3=NC(=NC=C3)N)NN=C2N
CNC1=NC(=CN=C1)C2=CNC(=O)C(=C2)NC(=O)C3=CC=C(C=C3)N4CCCC4CN5CCCC5
C1=CC=C2C(=C1)NC(=C(C#N)C3=NC(=NC=C3)NCCC4=CN=CC=C4)S2.C(=O)(C(F)(F)F)O.C(=O)(C(F)(F)F)O
CC(C)(C)C1=CC(=C(S1)NC(=O)NC2=C(C(=CC=C2)Cl)Cl)C(=O)N3CCC(=O)N(CC3)CCN(C)C
CC1CCC(CN1)N2CCC3(C2)CN(C4=CC=CC=C34)C(=O)C5=CC6=C(N5)C=C(C=C6)F
COC1=C(C=CC(=C1)C2=CC3=C(C=C2)C(=CC4=CC=CN4)C(=O)N3)O
CN1CCN(C(C1)C(=O)N)CC2=C(C=C3C(=C2)C(=NC=N3)NC4=C(C(=CC=C4)Cl)F)OC
COC1=C(C=C2C(=C1)N=CN=C2NC3=C(C(=CC=C3)Cl)F)CN4CCCCC4C(=O)N


  1%|          | 17/2111 [00:00<00:44, 47.13it/s]

COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)CN4CCCC4C(=O)N
COC1=C(C=C2C(=C1)N=CN=C2NC3=C(C(=CC=C3)Cl)F)CN4CC(C4)C(=O)N
CN1CCC(CC1)(C(=O)N)N(C)CC2=C(C=C3C(=C2)C(=NC=N3)NC4=C(C(=CC=C4)Cl)F)OC
COC1CC(N(C1)CC2=C(C=C3C(=C2)C(=NC=N3)NC4=C(C(=CC=C4)Cl)F)OC)C(=O)N
COC1=C(C=C2C(=C1)N=CN=C2NC3=C(C(=CC=C3)Cl)F)CN4CCC4C(=O)N
CN(CC1=C(C=C2C(=C1)C(=NC=N2)NC3=C(C(=CC=C3)Cl)F)OC)C(CCOC)C(=O)N
COC1=C(C=C2C(=C1)N=CN=C2NC3=C(C(=CC=C3)Cl)F)CN4CCCC4C(=O)N
COC1=C(C=C2C(=C1)N=CN=C2NC3=C(C(=CC=C3)Cl)F)CN4CCOCC4C(=O)N
CC(C(=O)N)N(C)CC1=C(C=C2C(=C1)C(=NC=N2)NC3=C(C(=CC=C3)Cl)F)OC
CN1CC(C1)(C(=O)N)N(C)CC2=C(C=C3C(=C2)C(=NC=N3)NC4=C(C(=CC=C4)Cl)F)OC
CN(CC1=C(C=C2C(=C1)C(=NC=N2)NC3=C(C(=CC=C3)Cl)F)OC)C4(CN(C4)CCOC)C(=O)N


...

COCCOCC#CC1=CC(=CS1)C2=NNC3=C2CC4=C3C=C(C=C4)CN5C=NC=N5
CN1C=C(N=C1)CNC2=CC(=C3C(=C2)C(=C(C=N3)C#N)NC4=CC(=C(C=C4)F)Cl)Br
CN1CCN(CC1)CC(=O)NC2=CC=CC3=C2CC4=C3NN=C4C5=CSC(=C5)C#CCOC6=CC=CC=C6
COCC(=O)NC1=CC=CC2=C1CC3=C2NN=C3C4=CSC(=C4)C#CCOC5=CC=CC=C5
COCCOCC(=O)NC1=CC=CC2=C1CC3=C2NN=C3C4=CSC(=C4)C#CCOC5=CC=CC=C5
COC(=O)C(=O)NC1=CC=CC2=C1CC3=C2NN=C3C4=CSC(=C4)C#CCOC5=CC=CC=C5
CC1=C(C=C(C=C1)O)NC2=NC(=NC=C2)NC3=CC=CC(=C3)C(=O)N
CN1CCN(CC1)CC2=CC3=C(C=C2)C4=C(C3)C(=NN4)C5=CSC(=C5)C#CCOC6=CC=CC=C6
CCOC(=O)CC(=O)NC1=CC=CC2=C1CC3=C2NN=C3C4=CSC(=C4)C#CCOC5=CC=CC=C5


100%|█████████▉| 2102/2111 [00:48<00:00, 45.51it/s]

CCOC(=O)C1=C2N=CC=C(N2N=C1)C3=CC(=CC=C3)NC(=O)C4=CC(=CC=C4)C(F)(F)F
C1C(C1NC2=NC(=O)C(=CC3=CC4=C(C=C3)N=CC=C4)S2)C5=CC=CC=C5
C1=CC2=C(C=CC(=C2)C=C3C(=O)N=C(S3)N)N=C1
CC(C)C1=C2N=C(C=C(N2N=C1)NCC3=CC=CC=C3)NC(CO)C(CO)O
CC(C)C1=C(C(=NN1)Cl)C2=NC(=NC=C2)NC3=NC=C(C=C3)N4CCC(CC4)N(C)C
CC(C)C1=C(C(=NN1)C(F)(F)F)C2=NC(=NC=C2)NC3=NC=C(C=C3)N4CCC(CC4)N(C)C
C1=CC=C(C=C1)C2=CC=NC3=C(C=NN23)C(=O)NC4=CC=C(C=C4)CO
C1=CC2=C(C=C1Br)C(=CN2)C3=NC(=NC=C3)N
C1=CN=C(N=C1C2=CNC3=CC(=CC(=C32)O)Br)N
C1=CC2=C(C=C1Br)NC=C2C3=NC(=NC=C3)N
C1=CC=C2C(=C1)C(=O)C3=C(C=CC(=C3C2=O)N)N
CC(C)(C)C1=NN(C(=C1)NC(=O)NC2=CC=CC3=CC=CC=C32)C4=CC=CC(=C4)C(=O)NCCCCN(C)C


100%|██████████| 2111/2111 [00:48<00:00, 43.40it/s]


C1=CC=C(C=C1)C2=CC=CN3C2=NNC3=O
CC1CC=CC(=O)CCCC=CC2=CC(=CC(=C2C(=O)O1)O)OC
CC1CC=CC(=O)CCCCCC2=CC(=CC(=C2C(=O)O1)O)OC
CCN1C2=C(C=CC(=C2)OC)SC1=CC(=O)C
CN1C=CC2=C1C3=C(C=C2)C4=C(C5=C(N4)C(=CC=C5)CCCNC6CCC(CC6)O)C7=C3C(=O)NC7=O
CCCC1=C(NC=N1)CNC2=CC(=C3C(=C2)C(=C(C=N3)C#N)NC4=CC(=C(C=C4)F)Cl)Cl
Here:  2111
Saved ligand features to /content/drive/MyDrive/Kd_Meshari/features/Kiba_ligand_features/Babel_Chemicals/ligand_descriptors.csv
(2111, 2546)
Feature extraction complete!


In [ ]:
!pip install -q mordred rdkit-pypi

In [ ]:
import pandas as pd
from rdkit import Chem
from mordred import Calculator, descriptors
from tqdm import tqdm
import numpy as np

# Step 1: Define excluded descriptor groups (already covered by your list)
excluded_groups = [
    "Basak", "Burden", "Pharmacophore", "Constitution", "Topology", "Connectivity", "Kappa", "EState",
    "Autocorrelation", "MolecularDistanceEdge", "Charge", "MOE", "MACCSKeys", "ECFP", "Estate"
]

# Step 2: Safely filter descriptors
calc = Calculator(descriptors, ignore_3D=True)

# Prepare an empty list for included descriptors
included_descriptors = []
print(len(calc.descriptors))
# Loop through all available descriptors
for d in calc.descriptors:
    module_path = d.__module__.split(".")

    # Check if the module path is deep enough to have a group identifier
    if len(module_path) > 1:
        group_name = module_path[1]
        # Exclude descriptors that belong to any of the specified groups
        if group_name not in excluded_groups:
            included_descriptors.append(d)
print(f"Number of descriptors selected: {len(included_descriptors)}")
calc = Calculator(included_descriptors, ignore_3D=True)


output_features_dir = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_ligand_features"
largest_fragments = "/content/drive/MyDrive/Kd_Meshari/features/kiba_ligands_processed1.csv"

# Load your input SMILES file
input_csv_path = largest_fragments  # ← Replace with your file path
df = pd.read_csv(input_csv_path)
print("Available columns in CSV:", df.columns.tolist())

# Confirm the SMILES column exists
if "smiles" not in df.columns:
    raise ValueError(f"Input CSV must contain a 'smiles' column. Available columns: {df.columns.tolist()}")

smiles_list = df["smiles"].tolist()
print(f"Processing {len(smiles_list)} SMILES...")

# Step 4: Calculate descriptors
results = []
failed = 0

for smi in tqdm(smiles_list):
    #print(smi)
    mol = Chem.MolFromSmiles(smi)

    if mol is None:
        failed += 1
        results.append([np.nan] * len(included_descriptors))
        continue
    try:
        desc = calc(mol)
        #print(desc)
        results.append([float(x) if x is not None else np.nan for x in desc])
    except Exception as e:
        failed += 1
        results.append([np.nan] * len(included_descriptors))

# Step 5: Convert to DataFrame and save
descriptor_names = [str(d) for d in included_descriptors]
features_df = pd.DataFrame(results, columns=descriptor_names)
features_df.insert(0, "smiles", smiles_list)  # keep SMILES for reference

output_csv = "/content/drive/MyDrive/Kd_Meshari/features/Kiba_ligand_features/mordred_additional_features.csv"
features_df.to_csv(output_csv, index=False)

print(f"✅ Saved: {output_csv}")
print(f"❌ Failed SMILES: {failed}")
print(f"Descriptor shape: {features_df.shape}")

1613
Number of descriptors selected: 672
Available columns in CSV: ['smiles']
Processing 2111 SMILES...


100%|██████████| 2111/2111 [06:00<00:00,  5.86it/s]


✅ Saved: /content/drive/MyDrive/Kd_Meshari/features/Kiba_ligand_features/mordred_additional_features.csv
❌ Failed SMILES: 0
Descriptor shape: (2111, 673)
